# For manipulating data and combining electron and cme data

In [1]:
import pandas as pd
from typing import Union

In [ ]:

def read_csv_file(file_path: str, delimiter: str = ',', header: Union[int, str] = 'infer', encoding: str = 'utf-8') -> pd.DataFrame:
    """
    Reads a CSV file and returns it as a pandas DataFrame.

    :param file_path: The path to the CSV file.
    :param delimiter: Character that separates the fields in the CSV file. Default is ','.
    :param header: Index of the row containing the header (column names), or 'infer' to infer it as the first line, or None if no header. Default is 'infer'.
    :param encoding: The encoding of the CSV file. Default is 'utf-8'.
    :return: A pandas DataFrame containing the content of the CSV file.
    """
    return pd.read_csv(file_path, delimiter=delimiter, header=header, encoding=encoding)
